## Setup libraries

In [1]:
from math import sqrt
import numpy as np
import pandas as pd
from pandasql import sqldf
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

# Setup tables

In [2]:
CremonaAir = read_csv("AirQuality1420.csv")
#Cremona = Cremona.loc[:, ["Date", "Ammonia", "PM10", "PM25",
#                       "Wind_speed", "Wind_direction", "Temperature", "Rainfall"]]


print(CremonaAir.shape)
CremonaAir.head()



(2548, 11)


,IDStation,NameStation,Date,Ammonia,CO,NO2,NOx,Ozone,PM10,PM25,Sulfur_dioxide
0,677,Cremona Via Fatebenefratelli,2014-01-01,2.5,0.700000,42.512500,85.045833,6.433333,73.0,66.0,2.504167
1,677,Cremona Via Fatebenefratelli,2014-01-02,4.5,0.595833,43.654167,80.879167,1.950000,51.0,NaN,1.800000
2,677,Cremona Via Fatebenefratelli,2014-01-03,1.6,0.470833,41.141667,60.845833,3.029167,47.0,46.0,1.870833
3,677,Cremona Via Fatebenefratelli,2014-01-04,1.1,0.487500,37.883333,63.150000,3.195833,42.0,38.0,1.050000
4,677,Cremona Via Fatebenefratelli,2014-01-05,0.7,0.520833,36.416667,57.283333,2.312500,21.0,21.0,0.854167


In [3]:
CremonaAdd= read_csv("RisultatiCremona.csv")
#Cremona = Cremona.loc[:, ["Date", "Ammonia", "PM10", "PM25",
#                       "Wind_speed", "Wind_direction", "Temperature", "Rainfall"]]

CremonaAdd = CremonaAdd.drop(["Unnamed: 0", "NameStation.1", "NameStation", "IDStation", "IDStation.1", "Date.1", "PM10", "Ammonia", "PM25"], axis='columns')
print(CremonaAdd.shape)
CremonaAdd.rename(columns={'Date': 'Date1'}, inplace=True)
CremonaAdd.head()

(2548, 7)


,Date1,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Rainfall
0,2014-01-01,NaN,NaN,2.233333,88.487500,56.600000,0.0
1,2014-01-02,NaN,NaN,3.658333,96.533333,9.841667,0.6
2,2014-01-03,NaN,NaN,4.029167,99.662500,12.537500,1.0
3,2014-01-04,NaN,NaN,4.691667,99.900000,4.316667,16.6
4,2014-01-05,NaN,NaN,5.412500,99.900000,7.887500,19.2


## Joining dataframe

In [4]:
Cremona = sqldf('''
SELECT  * 
FROM CremonaAir CA JOIN CremonaAdd CD
ON CA.Date = CD.Date1
''')

Cremona= Cremona.drop(["Date1"], axis='columns')

print(Cremona.shape)
Cremona.head()

(2548, 17)


,IDStation,NameStation,Date,Ammonia,CO,NO2,NOx,Ozone,PM10,PM25,Sulfur_dioxide,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Rainfall
0,677,Cremona Via Fatebenefratelli,2014-01-01,2.5,0.700000,42.512500,85.045833,6.433333,73.0,66.0,2.504167,NaN,NaN,2.233333,88.487500,56.600000,0.0
1,677,Cremona Via Fatebenefratelli,2014-01-02,4.5,0.595833,43.654167,80.879167,1.950000,51.0,NaN,1.800000,NaN,NaN,3.658333,96.533333,9.841667,0.6
2,677,Cremona Via Fatebenefratelli,2014-01-03,1.6,0.470833,41.141667,60.845833,3.029167,47.0,46.0,1.870833,NaN,NaN,4.029167,99.662500,12.537500,1.0
3,677,Cremona Via Fatebenefratelli,2014-01-04,1.1,0.487500,37.883333,63.150000,3.195833,42.0,38.0,1.050000,NaN,NaN,4.691667,99.900000,4.316667,16.6
4,677,Cremona Via Fatebenefratelli,2014-01-05,0.7,0.520833,36.416667,57.283333,2.312500,21.0,21.0,0.854167,NaN,NaN,5.412500,99.900000,7.887500,19.2


# Quadrants elaboration

In [5]:
#Primo quadrante
Q1 = Cremona.loc[((Cremona.Wind_direction > 0) &
                 (Cremona.Wind_direction < 90)), :]

a = Q1.shape[0]
arr1 = []
arr1 = [1 for i in range(a)]

del Q1['Wind_direction']
Q1["Wind_direction"] = arr1
columns_titles = ["index", "IDStation","NameStation","Date", "Ammonia", "CO", "NO2", "NOx","Ozone", "PM10", "PM25", "Sulfur_dioxide",
                 "Wind_speed",  "Temperature","Relative_humidity","Global_radiation", "Rainfall", "Wind_direction" ]
                  
Q1 = Q1.reindex(columns=columns_titles)
Q1 = Q1.drop("index", axis = 1)

#Secondo quadrante
Q2 = Cremona.loc[((Cremona.Wind_direction > 90) &
                 (Cremona.Wind_direction < 180)), :]
a = Q2.shape[0]
arr2 = []
arr2 = [2 for i in range(a)]

del Q2['Wind_direction']
Q2["Wind_direction"] = arr2

#Terzo quadrante
Q3 = Cremona.loc[((Cremona.Wind_direction > 180) &
                 (Cremona.Wind_direction < 270)), :]

a = Q3.shape[0]
arr3 = []
arr3 = [3 for i in range(a)]

del Q3['Wind_direction']
Q3["Wind_direction"] = arr3

#Quarto quadrante
Q4 = Cremona.loc[((Cremona.Wind_direction > 270) &
                 (Cremona.Wind_direction < 360)), :]

a = Q4.shape[0]
arr4 = []
arr4 = [4 for i in range(a)]

del Q4['Wind_direction']
Q4["Wind_direction"] = arr4

Cremona = sqldf('''
SELECT *
FROM Q1
UNION
SELECT *
FROM Q2
UNION
SELECT *
FROM Q3
UNION
SELECT *
FROM Q4
''')
#del Cremona['Wind_direction']
Cremona.rename(columns={'Wind_direction': 'Quadrante'}, inplace=True)

Cremona

/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_2956/3432512422.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q1["Wind_direction"] = arr1
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_2956/3432512422.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2["Wind_direction"] = arr2
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_2956/3432512422.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,IDStation,NameStation,Date,Ammonia,CO,NO2,NOx,Ozone,PM10,PM25,Sulfur_dioxide,Wind_speed,Temperature,Relative_humidity,Global_radiation,Rainfall,Quadrante
0,677,Cremona Via Fatebenefratelli,2014-02-12,6.5,0.550000,54.008333,149.795833,12.037500,54.0,44.0,3.112500,0.479634,8.004167,77.495833,124.250000,0.0,2
1,677,Cremona Via Fatebenefratelli,2014-02-13,8.3,0.566667,54.670833,147.558333,3.470833,52.0,43.0,1.375000,1.035337,5.375000,96.087500,31.375000,0.6,1
2,677,Cremona Via Fatebenefratelli,2014-02-14,7.7,0.587500,59.541667,153.525000,15.495833,43.0,36.0,2.070833,1.061836,7.508333,74.600000,118.258333,0.0,4
3,677,Cremona Via Fatebenefratelli,2014-02-15,9.6,0.429167,48.066667,84.470833,4.958333,41.0,35.0,2.591667,0.452648,6.404167,88.008333,13.620833,0.0,4
4,677,Cremona Via Fatebenefratelli,2014-02-16,7.4,0.587500,53.566667,87.491667,6.316667,67.0,59.0,2.991667,0.771591,7.554167,90.237500,38.241667,0.2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2465,677,Cremona Via Fatebenefratelli,2020-12-27,7.3,0.641667,22.879167,54.966667,9.462500,35.0,31.0,0.804167,0.045293,0.952083,92.725000,58.411806,0.0,1
2466,677,Cremona Via Fatebenefratelli,2020-12-28,6.7,0.529167,24.304167,38.829167,21.162500,NaN,20.0,0.616667,0.212185,0.277465,99.597183,8.239437,0.0,4
2467,677,Cremona Via Fatebenefratelli,2020-12-29,6.9,0.829167,46.133333,104.417391,2.608333,46.0,45.0,0.175000,0.451607,-0.663889,99.598611,28.943750,0.0,4
2468,677,Cremona Via Fatebenefratelli,2020-12-30,7.3,0.891667,49.329167,111.612500,2.516667,62.0,60.0,0.150000,0.634008,1.771528,99.568056,29.232639,0.0,3


In [6]:
Cremona

,IDStation,NameStation,Date,Ammonia,CO,NO2,NOx,Ozone,PM10,PM25,Sulfur_dioxide,Wind_speed,Temperature,Relative_humidity,Global_radiation,Rainfall,Quadrante
0,677,Cremona Via Fatebenefratelli,2014-02-12,6.5,0.550000,54.008333,149.795833,12.037500,54.0,44.0,3.112500,0.479634,8.004167,77.495833,124.250000,0.0,2
1,677,Cremona Via Fatebenefratelli,2014-02-13,8.3,0.566667,54.670833,147.558333,3.470833,52.0,43.0,1.375000,1.035337,5.375000,96.087500,31.375000,0.6,1
2,677,Cremona Via Fatebenefratelli,2014-02-14,7.7,0.587500,59.541667,153.525000,15.495833,43.0,36.0,2.070833,1.061836,7.508333,74.600000,118.258333,0.0,4
3,677,Cremona Via Fatebenefratelli,2014-02-15,9.6,0.429167,48.066667,84.470833,4.958333,41.0,35.0,2.591667,0.452648,6.404167,88.008333,13.620833,0.0,4
4,677,Cremona Via Fatebenefratelli,2014-02-16,7.4,0.587500,53.566667,87.491667,6.316667,67.0,59.0,2.991667,0.771591,7.554167,90.237500,38.241667,0.2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2465,677,Cremona Via Fatebenefratelli,2020-12-27,7.3,0.641667,22.879167,54.966667,9.462500,35.0,31.0,0.804167,0.045293,0.952083,92.725000,58.411806,0.0,1
2466,677,Cremona Via Fatebenefratelli,2020-12-28,6.7,0.529167,24.304167,38.829167,21.162500,NaN,20.0,0.616667,0.212185,0.277465,99.597183,8.239437,0.0,4
2467,677,Cremona Via Fatebenefratelli,2020-12-29,6.9,0.829167,46.133333,104.417391,2.608333,46.0,45.0,0.175000,0.451607,-0.663889,99.598611,28.943750,0.0,4
2468,677,Cremona Via Fatebenefratelli,2020-12-30,7.3,0.891667,49.329167,111.612500,2.516667,62.0,60.0,0.150000,0.634008,1.771528,99.568056,29.232639,0.0,3


# Exporting data

In [7]:
Cremona.to_csv(
    '/Users/marcovinciguerra/Github/GitTesi/Python/Cremona/QuadrantiNuovo/QuadrantiCremona2.csv')

